### Train a deep MLP on the MNIST dataset

In [1]:
from tensorflow import keras

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()

In [3]:
# Scale the features for Gradient Descent (is it really necessary?)
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.0

In [4]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(200, activation="relu"))
model.add(keras.layers.Dense(200, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))

2022-02-20 10:22:35.814423: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-02-20 10:22:35.816218: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-20 10:22:35.819625: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 200)               157000    
_________________________________________________________________
dense_1 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2010      
Total params: 199,210
Trainable params: 199,210
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(optimizer=keras.optimizers.SGD(),
              loss=keras.losses.sparse_categorical_crossentropy,
              metrics=keras.metrics.sparse_categorical_accuracy)

In [7]:
# Train the model with checkpoints and early stopping using callbacks
checkpoint_cb = keras.callbacks.ModelCheckpoint("mnist_ann.h5", save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=100,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb, early_stopping_cb])

2022-02-20 10:31:06.960467: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-02-20 10:31:06.989922: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2099940000 Hz


Epoch 1/100
1719/1719 [==============================] - 27s 15ms/step - loss: 1.1195 - sparse_categorical_accuracy: 0.6981 - val_loss: 0.3185 - val_sparse_categorical_accuracy: 0.9082
Epoch 2/100
1719/1719 [==============================] - 22s 13ms/step - loss: 0.3229 - sparse_categorical_accuracy: 0.9059 - val_loss: 0.2452 - val_sparse_categorical_accuracy: 0.9326
Epoch 3/100
1719/1719 [==============================] - 31s 18ms/step - loss: 0.2569 - sparse_categorical_accuracy: 0.9272 - val_loss: 0.2094 - val_sparse_categorical_accuracy: 0.9426
Epoch 4/100
1719/1719 [==============================] - 28s 16ms/step - loss: 0.2203 - sparse_categorical_accuracy: 0.9376 - val_loss: 0.1851 - val_sparse_categorical_accuracy: 0.9476
Epoch 5/100
1719/1719 [==============================] - 33s 19ms/step - loss: 0.1915 - sparse_categorical_accuracy: 0.9462 - val_loss: 0.1595 - val_sparse_categorical_accuracy: 0.9574
Epoch 6/100
1719/1719 [==============================] - 34s 20ms/step - lo

In [13]:
# saved model is the same as the `model` currently in memory as early_stopping_cb automatically
# rolls back to best model
model.evaluate(X_test, y_test)

313/313 [==============================] - 2s 6ms/step - loss: 0.0646 - sparse_categorical_accuracy: 0.9797


[0.06462757289409637, 0.9797000288963318]